# Interactive Serbian Trade Data

This is a project based on the Statistical Office of the Republic of Serbia open trade statistics dataset. It aims to provide an intuitive and direct way of interacting with the underlying data, enabling the user to ask questions and get a visual response.

The application is based on Dash, which on the other hand, relies on Plotly, React and Flask.

The making of this app is documented on my site, while the Jupyter Notebook with the data wrangling is here.

Data wrangling for Dash
In this notebook I have played around with the Statistical Office Open Dataset, did some data wrangling and simple transformations and so on.

I will be working with two, possibly three datasets, each with an increasing level of granularity. They do share, however, a similar structure: each file has some columns in common: the year, the country, the name of the commodity group, codes for country and group and so on. The rows contain two groups of data: quantity in kg and the value in US dollars.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import os.path

## The simple(r) dataset

The first dataset contains just around 30.000 rows and describes just the topmost 10 trade categories. The SMTK category system, as I learned, is based on a number or cipher hierarchy. For simplicity, let's say that the most basic categories are denoted with numbers 0 to 9. The next level is composed of two digits, where 12 and 11, for example, belong to the 1-category and so on...

Since I cannot put the datasets on GitHub, they are on Dropbox in a public folder. Just make sure to put dl=1 at the end, instead of the default dl=0.

In [104]:
serbian_imports = pd.read_csv('https://www.dropbox.com/s/s06v0k9vpvjp5sf/imp_groups.csv?dl=0', sep=';')

The **read_csv** went smooth, as we'll see later, this isn't the case for the bigger datasets which contain errors.

In [105]:
serbian_imports.head()

,idindikator,mes,god,IDDrzave,nDrzave,IDNSST,nNSST,IDVrPod,nVrPod,vrednost
0,170301IND02,0,2004,00,All countries,0,Food and live animals,1,"Quantity, tonnes",823033.1
1,170301IND02,0,2004,00,All countries,0,Food and live animals,2,"Total, USD thousand",624482.5
2,170301IND02,0,2004,00,All countries,1,Beverages and tobacco,1,"Quantity, tonnes",51928.9
3,170301IND02,0,2004,00,All countries,1,Beverages and tobacco,2,"Total, USD thousand",161663.0
4,170301IND02,0,2004,00,All countries,2,"Crude materials,inedible,except fuels",1,"Quantity, tonnes",2398144.2


In [4]:
serbian_imports.dtypes

idindikator     object
mes              int64
god              int64
IDDrzave        object
nDrzave         object
IDNSST           int64
nNSST           object
IDVrPod          int64
nVrPod          object
vrednost       float64
dtype: object

Again, here the category ID (IDNSST) is parsed as an integer, since it can take values from 0-9. However, for the bigger dataset that we'll examine later - this will not work. 

In [5]:
# check integrity - there should be only two possible values
serbian_imports.nVrPod.unique()

array(['Quantity, tonnes', 'Total, USD thousand'], dtype=object)

In [6]:
serbian_imports.nDrzave.unique()

array(['All countries', 'Andorra', 'United Arab Emirates', 'Afghanistan',
       'Antigua and Barbuda', 'Albania', 'Armenia',
       'Netherlands Antilles', 'Angola', 'Argentina', 'Austria',
       'Australia', 'Azerbaijan', 'Rep. of Bosnia and Herzegovina',
       'Barbados', 'Bangladesh', 'Belgium', 'Burkina Faso', 'Bulgaria',
       'Bahrain', 'Burundi', 'Benin', 'Bermuda', 'Brunei', 'Bolivia',
       'Brazil', 'Bahamas', 'Bhutan', 'Botswana', 'Belarusa', 'Belize',
       'Canada', 'Central African Republic', 'Congo', 'Switzerland',
       'Ivory Coast', 'Cook Islands', 'Chile', 'Cameroon', 'China',
       'Columbia', 'Costa Rica', 'Cuba', 'Cape Verde Islands',
       'Christmas Island', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Denmark', 'Dominica', 'Dominican Republic', 'Algeria',
       'Ecuador', 'Estonia', 'Egypt', 'Eritrea', 'Spain', 'Ethiopia',
       'European Monetary Union', 'Finland', 'Falkland Islands',
       'Faroe Islands', 'France', 'Gabon', 'United 

Not all of these are countries (EU, CEFTA etc) - we'll filter them out.

In [7]:
serbian_imports.IDDrzave.unique()

array(['00', 'AD', 'AE', 'AF', 'AG', 'AL', 'AM', 'AN', 'AO', 'AR', 'AT',
       'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ',
       'BM', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CF',
       'CG', 'CH', 'CI', 'CK', 'CL', 'CM', 'CN', 'CO', 'CR', 'CU', 'CV',
       'CX', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE',
       'EG', 'ER', 'ES', 'ET', 'EU', 'FI', 'FK', 'FO', 'FR', 'GA', 'GB',
       'GD', 'GE', 'GF', 'GH', 'GI', 'GL', 'GM', 'GN', 'GQ', 'GR', 'GS',
       'GT', 'GU', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN',
       'IO', 'IQ', 'IR', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH',
       'KI', 'KM', 'KN', 'KP', 'KR', 'KW', 'KY', 'KZ', 'LA', 'LB', 'LI',
       'LK', 'LR', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD', 'MG', 'MH',
       'MK', 'ML', 'MM', 'MN', 'MO', 'MP', 'MR', 'MT', 'MU', 'MV', 'MW',
       'MX', 'MY', 'MZ', nan, 'NC', 'NE', 'NG', 'NI', 'NL', 'NO', 'NP',
       'NR', 'NU', 'NZ', 'OM', 'PA', 'PE', 'PF', 'PG

As I found out, unfortunately Plotly doesn't like these __two letter__ country codes - it needs the three letter code in order to display the maps. I used a package called _pycountry_, but there are others as well.

In [8]:
serbian_exports = pd.read_csv('https://www.dropbox.com/s/0w8ptgmmdy7ok6w/exp_groups.csv?dl=1', sep=';')

In [9]:
# again - I'm using head all the time, because I am too lazy to remember anything really...
serbian_imports.head()

,idindikator,mes,god,IDDrzave,nDrzave,IDNSST,nNSST,IDVrPod,nVrPod,vrednost
0,170301IND02,0,2004,00,All countries,0,Food and live animals,1,"Quantity, tonnes",823033.1
1,170301IND02,0,2004,00,All countries,0,Food and live animals,2,"Total, USD thousand",624482.5
2,170301IND02,0,2004,00,All countries,1,Beverages and tobacco,1,"Quantity, tonnes",51928.9
3,170301IND02,0,2004,00,All countries,1,Beverages and tobacco,2,"Total, USD thousand",161663.0
4,170301IND02,0,2004,00,All countries,2,"Crude materials,inedible,except fuels",1,"Quantity, tonnes",2398144.2


In [10]:
serbian_exports.head()

,idindikator,mes,god,IDDrzave,nDrzave,IDNSST,nNSST,IDVrPod,nVrPod,vrednost
0,170301IND01,0,2004,00,All countries,0,Food and live animals,1,"Quantity, tonnes",1121749.8
1,170301IND01,0,2004,00,All countries,0,Food and live animals,2,"Total, USD thousand",641392.0
2,170301IND01,0,2004,00,All countries,1,Beverages and tobacco,1,"Quantity, tonnes",115060.7
3,170301IND01,0,2004,00,All countries,1,Beverages and tobacco,2,"Total, USD thousand",55802.2
4,170301IND01,0,2004,00,All countries,2,"Crude materials,inedible,except fuels",1,"Quantity, tonnes",583512.1


We don't need all of the columns:

- the mes (month) column is always 0 which means it is reffered to the 12 month period - an entire year, so it doesn't yield us any information
- IDDrzave (country ID) is kinda redundant as we will use the full name of the countries (nDrzave), but we will need it in order to display the world map. First, however, we'll need to transform it to a 3-letter code (also called APLHA code)
- we're keeping IDVrPod as it gives us a nice way to filter out the quantities in kg (which we do not need) and values in dollars which we wish to keep and work on
- vrednost - it's the value that we wish to keep (in USD)

In [11]:
# columns to keep
columns_to_keep = ['god','nDrzave','IDNSST','nNSST','IDDrzave','IDVrPod','vrednost']

In [12]:
# keep only the desired columns
serbian_imports = serbian_imports[columns_to_keep]
serbian_exports = serbian_exports[columns_to_keep]

In [13]:
serbian_exports.head(5)

,god,nDrzave,IDNSST,nNSST,IDDrzave,IDVrPod,vrednost
0,2004,All countries,0,Food and live animals,00,1,1121749.8
1,2004,All countries,0,Food and live animals,00,2,641392.0
2,2004,All countries,1,Beverages and tobacco,00,1,115060.7
3,2004,All countries,1,Beverages and tobacco,00,2,55802.2
4,2004,All countries,2,"Crude materials,inedible,except fuels",00,1,583512.1


Now we'll rename the columns - keeping it all in English.

In [14]:
serbian_imports.rename(columns={'god':'year',
                                'nDrzave':'country',
                                'IDDrzave':'ccode',
                                'nNSST':'category',
                                'vrednost':'import_value'
                               }, inplace=True);
serbian_exports.rename(columns={'god':'year',
                                'nDrzave':'country',
                                'IDDrzave':'ccode',
                                'nNSST':'category',
                                'vrednost':'export_value'
                               }, inplace=True);

In [15]:
serbian_exports.head(5)

,year,country,IDNSST,category,ccode,IDVrPod,export_value
0,2004,All countries,0,Food and live animals,00,1,1121749.8
1,2004,All countries,0,Food and live animals,00,2,641392.0
2,2004,All countries,1,Beverages and tobacco,00,1,115060.7
3,2004,All countries,1,Beverages and tobacco,00,2,55802.2
4,2004,All countries,2,"Crude materials,inedible,except fuels",00,1,583512.1


In [16]:
serbian_imports.head(5)

,year,country,IDNSST,category,ccode,IDVrPod,import_value
0,2004,All countries,0,Food and live animals,00,1,823033.1
1,2004,All countries,0,Food and live animals,00,2,624482.5
2,2004,All countries,1,Beverages and tobacco,00,1,51928.9
3,2004,All countries,1,Beverages and tobacco,00,2,161663.0
4,2004,All countries,2,"Crude materials,inedible,except fuels",00,1,2398144.2


This is probably the most important, albeit simple step: **merging the two dataframes into one**. The only thing to remember is to always include **all** the column names that are in common.

In [17]:
commerce = pd.merge(serbian_imports,serbian_exports ,  how='left',
                    left_on=['year','country', 'category','IDVrPod','ccode','IDNSST'],
                    right_on =['year','country', 'category','IDVrPod','ccode','IDNSST'])

In [18]:
commerce.head()

,year,country,IDNSST,category,ccode,IDVrPod,import_value,export_value
0,2004,All countries,0,Food and live animals,00,1,823033.1,1121749.8
1,2004,All countries,0,Food and live animals,00,2,624482.5,641392.0
2,2004,All countries,1,Beverages and tobacco,00,1,51928.9,115060.7
3,2004,All countries,1,Beverages and tobacco,00,2,161663.0,55802.2
4,2004,All countries,2,"Crude materials,inedible,except fuels",00,1,2398144.2,583512.1


Later we'll do some testing to see if the numbers correspond with the official reports issued by the Office.

Let's filter out all the rows with quantities in kg and leave only the value in dollars.

In [19]:
commerce = commerce[commerce['IDVrPod']==2]

In [20]:
commerce.head(8)

,year,country,IDNSST,category,ccode,IDVrPod,import_value,export_value
1,2004,All countries,0,Food and live animals,00,2,624482.5,641392.0
3,2004,All countries,1,Beverages and tobacco,00,2,161663.0,55802.2
5,2004,All countries,2,"Crude materials,inedible,except fuels",00,2,332391.0,195490.4
7,2004,All countries,3,"Mineral fuels,lubricants and related materials",00,2,1633635.3,90841.0
9,2004,All countries,4,"Animal and vegetable oils,fats and waxes",00,2,15708.4,58874.2
11,2004,All countries,5,"Chemicals and related products,not elsewhere s...",00,2,1409608.3,387122.8
13,2004,All countries,6,Manufactured goods classified chiefly by material,00,2,2088717.1,1184861.5
15,2004,All countries,7,Machinery and transport equipment,00,2,3381880.6,377716.3


In [21]:
# drop the IDVRPOD - we dont need it anymore
commerce.drop('IDVrPod',axis=1, inplace=True)

In [22]:
commerce.head(2)

,year,country,IDNSST,category,ccode,import_value,export_value
1,2004,All countries,0,Food and live animals,00,624482.5,641392.0
3,2004,All countries,1,Beverages and tobacco,00,161663.0,55802.2


Now... I know these values are so simple that it seems stupid, but I believe it is simpler and maybe more efficient to just include the calculatred values in the source CSv than to have it calculated each time the server is up...

In [23]:
#add the trade balance and total exchange
commerce['balance'] = commerce['export_value']-commerce['import_value']
commerce['exchange'] = commerce['export_value'] + commerce['import_value']

In [24]:
commerce.head()

,year,country,IDNSST,category,ccode,import_value,export_value,balance,exchange
1,2004,All countries,0,Food and live animals,00,624482.5,641392.0,16909.5,1265874.5
3,2004,All countries,1,Beverages and tobacco,00,161663.0,55802.2,-105860.8,217465.2
5,2004,All countries,2,"Crude materials,inedible,except fuels",00,332391.0,195490.4,-136900.6,527881.4
7,2004,All countries,3,"Mineral fuels,lubricants and related materials",00,1633635.3,90841.0,-1542794.3,1724476.3
9,2004,All countries,4,"Animal and vegetable oils,fats and waxes",00,15708.4,58874.2,43165.8,74582.6


Now it is time to clean up the dataset by removing all the rows corresponding to non-countries like the EU, CEFTA, All countries and so on...

In [25]:
commerce.country.unique()

array(['All countries', 'Andorra', 'United Arab Emirates', 'Afghanistan',
       'Antigua and Barbuda', 'Albania', 'Armenia',
       'Netherlands Antilles', 'Angola', 'Argentina', 'Austria',
       'Australia', 'Azerbaijan', 'Rep. of Bosnia and Herzegovina',
       'Barbados', 'Bangladesh', 'Belgium', 'Burkina Faso', 'Bulgaria',
       'Bahrain', 'Burundi', 'Benin', 'Bermuda', 'Brunei', 'Bolivia',
       'Brazil', 'Bahamas', 'Bhutan', 'Botswana', 'Belarusa', 'Belize',
       'Canada', 'Central African Republic', 'Congo', 'Switzerland',
       'Ivory Coast', 'Cook Islands', 'Chile', 'Cameroon', 'China',
       'Columbia', 'Costa Rica', 'Cuba', 'Cape Verde Islands',
       'Christmas Island', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Denmark', 'Dominica', 'Dominican Republic', 'Algeria',
       'Ecuador', 'Estonia', 'Egypt', 'Eritrea', 'Spain', 'Ethiopia',
       'European Monetary Union', 'Finland', 'Falkland Islands',
       'Faroe Islands', 'France', 'Gabon', 'United 

In [26]:
# this is manual labour, visual incpection... :(
not_countries = [
    'All countries',
    'CEFTA countries',
    'Countries not specified in extra',
    'European Monetary Union',
    'European Union (28)'   
    ]

In [27]:
commerce = commerce[commerce['country'].isin(not_countries)==False]

In [28]:
commerce.head()

,year,country,IDNSST,category,ccode,import_value,export_value,balance,exchange
21,2004,Andorra,0,Food and live animals,AD,27.0,0.0,-27.0,27.0
23,2004,Andorra,5,"Chemicals and related products,not elsewhere s...",AD,0.1,0.0,-0.1,0.1
25,2004,Andorra,6,Manufactured goods classified chiefly by material,AD,4.8,0.0,-4.8,4.8
27,2004,Andorra,7,Machinery and transport equipment,AD,24.4,0.0,-24.4,24.4
29,2004,Andorra,8,Miscellaneous manufactured articles,AD,120.8,0.0,-120.8,120.8


Finally, let's convert the country codes and get the three-letter representation. Enter pycountry.

In [29]:
import pycountry
def convert_ccode(code):
    try:
        country = pycountry.countries.get(alpha_2=code)
        return country.alpha_3
    except AttributeError:
        return None

In [30]:
convert_ccode('DE')

'DEU'

In [31]:
convert_ccode('IT')

'ITA'

You get the idea...

In [32]:
# fix country code
commerce['ccode']=commerce['ccode'].apply(convert_ccode)

In [33]:
commerce.sample(10)

,year,country,IDNSST,category,ccode,import_value,export_value,balance,exchange
829,2004,Ghana,8,Miscellaneous manufactured articles,GHA,1.1,29.8,28.7,30.9
22093,2013,Armenia,5,"Chemicals and related products,not elsewhere s...",ARM,0.0,604.1,604.1,604.1
23119,2013,India,5,"Chemicals and related products,not elsewhere s...",IND,50814.7,2418.2,-48396.5,53232.9
11219,2008,Mozambique,2,"Crude materials,inedible,except fuels",MOZ,352.2,0.0,-352.2,352.2
30213,2016,Belgium,8,Miscellaneous manufactured articles,BEL,8531.9,15144.5,6612.6,23676.4
37077,2018,Luxembourg,7,Machinery and transport equipment,LUX,1880.3,2042.5,162.2,3922.8
3809,2005,Mali,3,"Mineral fuels,lubricants and related materials",MLI,0.6,0.0,-0.6,0.6
1081,2004,Italy,0,Food and live animals,ITA,26578.7,125489.4,98910.7,152068.1
13311,2009,Korea,6,Manufactured goods classified chiefly by material,KOR,5461.0,24.6,-5436.4,5485.6
21039,2012,Malta,1,Beverages and tobacco,MLT,0.0,89.1,89.1,89.1


In [34]:
# Let's check if any ccode has not exactly 3 characters:
commerce.isnull().any()

year            False
country         False
IDNSST          False
category        False
ccode            True
import_value    False
export_value    False
balance         False
exchange        False
dtype: bool

In [35]:
# OK, so it is the ccode... Let's investigate.
nan_rows = commerce[commerce['ccode'].isnull()]

In [36]:
nan_rows

,year,country,IDNSST,category,ccode,import_value,export_value,balance,exchange
97,2004,Netherlands Antilles,7,Machinery and transport equipment,None,0.0,0.1,0.1,0.1
1521,2004,Namibia,0,Food and live animals,None,293.4,0.0,-293.4,293.4
1523,2004,Namibia,2,"Crude materials,inedible,except fuels",None,2.9,0.0,-2.9,2.9
1525,2004,Namibia,6,Manufactured goods classified chiefly by material,None,0.0,3.0,3.0,3.0
1527,2004,Namibia,7,Machinery and transport equipment,None,2.8,2.9,0.1,5.7
1529,2004,Namibia,8,Miscellaneous manufactured articles,None,2.7,8.8,6.1,11.5
1531,2004,Namibia,9,SMTK Rev. 4 Commodities n.e.s. in the SITC Rev. 4,None,0.0,1.4,1.4,1.4
2131,2004,East Timor,6,Manufactured goods classified chiefly by material,None,0.1,0.0,-0.1,0.1
2317,2004,Palestine Liberation Organisat,2,"Crude materials,inedible,except fuels",None,0.0,19.0,19.0,19.0
2353,2004,Zaire,0,Food and live animals,None,0.6,30.8,30.2,31.4


In [37]:
# there aren't so many countries that cause pycountry to choke... 6 of them.
nan_rows.country.unique()

array(['Netherlands Antilles', 'Namibia', 'East Timor',
       'Palestine Liberation Organisat', 'Zaire', 'France, Metropolitan'],
      dtype=object)

### Decisions, decisions

Now, this is a perfect example of data in desperate need of a fix. This malfunction is so small, that it would be a shame to just  dismiss it with a dropna command.

In [38]:
# what was the initial value in the original file?
serbian_exports[serbian_exports['country']=='Namibia'].head()

,year,country,IDNSST,category,ccode,IDVrPod,export_value
1520,2004,Namibia,0,Food and live animals,NaN,1,0.0
1521,2004,Namibia,0,Food and live animals,NaN,2,0.0
1522,2004,Namibia,2,"Crude materials,inedible,except fuels",NaN,1,0.0
1523,2004,Namibia,2,"Crude materials,inedible,except fuels",NaN,2,0.0
1524,2004,Namibia,6,Manufactured goods classified chiefly by material,NaN,1,24.9


In [39]:
# Ok... so there was no code whatsoever. How about the others?
# what was the initial value in the original file?
serbian_exports[serbian_exports['country']=='East Timor'].head()

,year,country,IDNSST,category,ccode,IDVrPod,export_value
2130,2004,East Timor,6,Manufactured goods classified chiefly by material,TP,1,0.0
2131,2004,East Timor,6,Manufactured goods classified chiefly by material,TP,2,0.0
4494,2005,East Timor,5,"Chemicals and related products,not elsewhere s...",TP,1,0.0
4495,2005,East Timor,5,"Chemicals and related products,not elsewhere s...",TP,2,0.0
4496,2005,East Timor,6,Manufactured goods classified chiefly by material,TP,1,0.0


In the original file there was the code **NA** for Namibia, which is correct, so it's probably Pandas. 
After a quick [stackoverflow check](https://stackoverflow.com/questions/41417214/prevent-pandas-from-reading-na-as-nan) it turns out I am right.
I want to fix this now manually, and then we'll update the main processing ETL function later, including this NA hack.

In [40]:
# east Timor, however isn't recognized by pycountry
print(convert_ccode('TP'))

None


In [41]:
# neither is Zaire
print(convert_ccode('ZR'))

None


In [42]:
print(convert_ccode('XP'))

None


In [43]:
print(convert_ccode('PS'))

PSE


In [44]:
print(convert_ccode('AN'))

None


Decisions... Well, the decision here is the following set of rules:

 - Namibia gets converted to Namibia manually - NAM
 - Netherlands Antilles becomes Netherlands - NLD
 - East Timor gets converted manually to - TMP
 - Palestine Liberation Organisat becomes Palestine - PSE
 - Zaire gets converted to Zaire manually - ZAR
 - France, Metropolitan gets added to France - FRA

In [45]:
def fix_ccode(val):
    
    if 'Namibia' in val['country']:
        return 'NAM'
    if 'Netherlands Antilles' in val['country']:
        return 'NLD'
    if 'East Timor' in val['country']:
        return 'TMP'
    if 'Palestine' in val['country']:
        return 'PSE'
    if 'Zaire' in val['country']:
        return 'ZAR'
    if 'France, Metropolitan' in val['country']:
        return 'FRA'
    else:
        return val['ccode']

In [46]:
# fingers crossed...
commerce.head()

,year,country,IDNSST,category,ccode,import_value,export_value,balance,exchange
21,2004,Andorra,0,Food and live animals,AND,27.0,0.0,-27.0,27.0
23,2004,Andorra,5,"Chemicals and related products,not elsewhere s...",AND,0.1,0.0,-0.1,0.1
25,2004,Andorra,6,Manufactured goods classified chiefly by material,AND,4.8,0.0,-4.8,4.8
27,2004,Andorra,7,Machinery and transport equipment,AND,24.4,0.0,-24.4,24.4
29,2004,Andorra,8,Miscellaneous manufactured articles,AND,120.8,0.0,-120.8,120.8


In [47]:
commerce['ccode'] = commerce.apply(fix_ccode, axis=1)

In [48]:
commerce[commerce['country']=='East Timor']['ccode']

2131    TMP
4495    TMP
4497    TMP
4499    TMP
4501    TMP
Name: ccode, dtype: object

In [49]:
commerce.isnull().any()

year            False
country         False
IDNSST          False
category        False
ccode           False
import_value    False
export_value    False
balance         False
exchange        False
dtype: bool

Boring, but necessary.

## Testing testing

Now we'll get a couple of points and try to compare them with the _official_ data.


In [50]:
commerce.category.unique()

array(['Food and live animals',
       'Chemicals and related products,not elsewhere specified',
       'Manufactured goods classified chiefly by material',
       'Machinery and transport equipment',
       'Miscellaneous manufactured articles',
       'Crude materials,inedible,except fuels',
       'Mineral fuels,lubricants and related materials',
       'SMTK Rev. 4 Commodities n.e.s. in the SITC Rev. 4',
       'Beverages and tobacco',
       'Animal and vegetable oils,fats and waxes'], dtype=object)

In [51]:
# let's pick some
commerce[(commerce['year']==2018)&(commerce['country']=='Italy')]['import_value'].sum()

2422626.6999999997

In [52]:
commerce[(commerce['year']==2018)&(commerce['country']=='Poland')]['import_value'].sum()

921663.3

In [53]:
commerce[(commerce['year']==2018)&(commerce['country']=='Austria')]['import_value'].sum()

720491.5

In [54]:
commerce[(commerce['year']==2018)&(commerce['country']=='Italy')]['export_value'].sum()

2356420.8

In [55]:
commerce[(commerce['year']==2018)&(commerce['country']=='France')]['export_value'].sum()

530213.6000000001

In [56]:
commerce[(commerce['year']==2018)&(commerce['country']=='Iraq')]['export_value'].sum()

9370.5

In [57]:
commerce[(commerce['year']==2018)&(commerce['country']=='Sweden')]['export_value'].sum()

203207.9

In [58]:
commerce[(commerce['year']==2017)&(commerce['country']=='Italy')]['export_value'].sum()

2237025.0000000005

In [59]:
commerce[(commerce['year']==2017)&(commerce['country']=='China')]['export_value'].sum()

62160.2

In [60]:
commerce[(commerce['year']==2018)&(commerce['category']=='Food and live animals')]['export_value'].sum()

2419310.7

In [61]:
commerce[(commerce['year']==2017)&(commerce['category']=='Food and live animals')]['export_value'].sum()

2291579.8

In [62]:
commerce[(commerce['year']==2018)&(commerce['category']=='Beverages and tobacco')]['import_value'].sum()

344669.60000000003

In [63]:
commerce[(commerce['year']==2018)&(commerce['category']=='Mineral fuels,lubricants and related materials')]['export_value'].sum()

571385.8999999999

In [64]:
commerce[commerce['year']==2018]['import_value'].sum()

25843009.700000003

In [65]:
commerce[commerce['year']==2018]['export_value'].sum()

19226475.1

In [66]:
commerce[commerce['year']==2017]['import_value'].sum()

21915344.799999997

In [67]:
commerce[(commerce['year']==2017)&(commerce['IDNSST']==2)]['export_value'].sum()

599235.9

In [68]:
commerce[(commerce['year']==2018)&(commerce['IDNSST']==5)]['import_value'].sum()

3507984.5

I know it looks ugly, but this is manual testing, with pen and paper. It looks ok, bearing in mind that the printed report I used for comparisons is 3-4 months older than the live open data that we are using here - fluctuations and corrections are expected.

## The bigger dataset

We could wrap it up and create the ETL function that reads the raw csv files, for exports and imports, but bz doing so we would fail to reuse this code for the bigger datasets - the one describing the 3digits categories.

### We re gonna need a bigger boat

This is going to be just a continuation of the first part - I want to simply replace the existing data with a bigger dataset. While in the forst part I worked with the SITC REV. 4 SECTIONS, now we'll dive a bit deeper, using the so-called SITC REV. 4 GROUPS. The first datasets had 38350 records each, while this has 593538 records: almost 20 times more.

This doesn't work...

exports = pd.read_csv('exp_groups.csv', sep=';')

Turns out that some categories have a ";" symbol inside the description. Now, since ";" is used as a delimiter, this gives us a variable number of fields and the pandas parser, rightfully so, throws an error. At first I tried to replace all the occurences of the error manually, but then I understood that the error is systematic and general: the extra semicolons appear, if they appear, only in the 7th field. Let's see a typical example.

In [69]:
row = "170303IND01;00;2004;00;All countries;03;Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates;preparations thereof;1;Quantity, tonnes;172.7"

When we identify the error in this way, we can reuse the parts of the row that are **always** good: the first 6 fields and the last 3.

In [70]:
lst = row.split(";")

In [71]:
lst

['170303IND01',
 '00',
 '2004',
 '00',
 'All countries',
 '03',
 'Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates',
 'preparations thereof',
 '1',
 'Quantity, tonnes',
 '172.7']

In [72]:
# the first 6 are ok
beginning =lst[:6]
print(beginning)

['170303IND01', '00', '2004', '00', 'All countries', '03']


In [73]:
# the last 3 as well
end =lst[-3:]

In [74]:
end

['1', 'Quantity, tonnes', '172.7']

In [75]:
# the (potentially) bad part
middle = lst[6:][:-3]

In [76]:
middle

['Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates',
 'preparations thereof']

In [77]:
# now i just join the middle to a string
middle = ','.join(middle)

In [78]:
middle

'Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates,preparations thereof'

In [79]:
# and sew this all back
beginning.append(middle)

In [80]:
good_row = beginning+end

In [81]:
good_row

['170303IND01',
 '00',
 '2004',
 '00',
 'All countries',
 '03',
 'Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates,preparations thereof',
 '1',
 'Quantity, tonnes',
 '172.7']

In [82]:
print(';'.join(good_row))

170303IND01;00;2004;00;All countries;03;Fish(not marine mammals),crustaceans,molluscs and aquatic invertebrates,preparations thereof;1;Quantity, tonnes;172.7


Wrapping this in simple function:

In [83]:
def fix_csv(old_csv, new_csv):
    # function for converting the problematic csv file
    
    with open(old_csv,"r") as input_file: 
        with open(new_csv, "w") as output_file:
                      
            for line in input_file:
                _fields = line.split(";")
                _start =_fields[:6]
                _end = _fields[-3:]
                _middle = ",".join(_fields[6:][:-3])
                _start.append(_middle)
                new_line = _start+_end
                output_file.write(';'.join(new_line)+'\n')
    print('File {} written...'.format(new_csv))
    return new_csv

In [108]:
# fingers crossed!
fix_csv('exp_groups.csv','cleaned_exp.csv')

File cleaned_exp.csv written...


'cleaned_exp.csv'

In [ ]:
df = pd.read_csv('cleaned_exp.csv',sep=';')

It works. Now let's see the differences, the category depth and so on.

In [ ]:
df.head()

In [ ]:
df.IDNSST.unique()

In [ ]:
len(df.IDNSST.unique())

In [ ]:
len(df.nNSST.unique())

### Danger!

Wow! These two value counts should be identical! What happened?
As it turned out, the category ID **are not numbers** - they are IDs alright, but not numeric. They follow a hierarchy where the topmost category 0 contains subcategories like 00, 01 etc. We cannot simply load the csv files, since pandas will read all IDNSST fields as integers, be it 1 or 01 or 001, converting them to 1 uncoditionally.

What we need to do is read the CSV file whilst preserving the IDNSST field as a string: dtype={'IDNSST': object}

In [90]:
df = pd.read_csv('cleaned_exp.csv',sep=';', dtype={'IDNSST': object})

In [91]:
len(df.IDNSST.unique())

337

In [92]:
len(df.nNSST.unique())

329

Here, we are interested **only** in the codes that contain exactly three digits, not two, not one.

In [93]:
mask = (df['IDNSST'].str.len() == 3)
df = df.loc[mask]

In [94]:
len(df.IDNSST.unique())

261

In [95]:
len(df.nNSST.unique())

261

## The Big Function

Here I'll try to wrap everything learnt so far, in a big fat function that should take in two raw csv files - one for the exports and the other one for the imports - do all the processing, joining, filtering and so on and output a new csv file, ready to be visualized with plotly and Dash.

In [96]:
# code the function
def preprocess(export_file, import_file, output_file='processed.csv'):
    
    columns_to_keep = ['god','nDrzave','nNSST','IDNSST','IDDrzave','IDVrPod','vrednost']
    
    not_countries = [
    'All countries',
    'CEFTA countries',
    'Countries not specified in extra',
    'European Monetary Union',
    'European Union (28)'   
    ]
    
    # if the csvs have been fixed, don't bother doing it again
    if not (os.path.exists('fixed_export_file.csv') and os.path.exists('fixed_import_file.csv')):
    
        # fix csv files
        fix_csv(export_file, 'fixed_export_file.csv')
        fix_csv(import_file, 'fixed_import_file.csv')
    
    # load in pandas, separator is ";" and IDNSST is a STRING!
    exp = pd.read_csv('fixed_export_file.csv',sep=";",dtype={'IDNSST': object})
    imp = pd.read_csv('fixed_import_file.csv',sep=";", dtype={'IDNSST': object})
    
    exp = exp[columns_to_keep]
    imp = imp[columns_to_keep]
    
    # columns
    exp.rename(columns={
        'god':'year',
        'nDrzave':'country',
        'IDDrzave':'ccode',
        'nNSST':'category',
        'vrednost':'export_value'
    }, inplace=True)
    
    imp.rename(columns={
        'god':'year',
        'nDrzave':'country',
        'IDDrzave':'ccode',
        'nNSST':'category',
        'vrednost':'import_value'
    }, inplace=True)
    
     
    # merge
    commerce = pd.merge(imp, exp, how='left',
                        left_on=['year','country', 'category','IDVrPod','ccode','IDNSST'],
                        right_on =['year','country', 'category','IDVrPod','ccode','IDNSST'])
    
    # filter rows
    commerce = commerce[commerce['IDVrPod']==2]
    
    # filter IDNSST not in 0-9 AS STRINGS OF LENGTH 3!    
    mask = (commerce['IDNSST'].str.len() == 3)
    commerce = commerce.loc[mask]
      
    
    # drop IDVrPod
    commerce.drop('IDVrPod',axis=1, inplace=True)
    
    # remove non countries
    commerce = commerce[commerce['country'].isin(not_countries)==False]
    
    # fix country code
    commerce['ccode']=commerce['ccode'].apply(convert_ccode)
    
    #add the trade balance and total exchange
    commerce['balance'] = commerce['export_value']-commerce['import_value']
    commerce['exchange'] = commerce['export_value'] + commerce['import_value']
    
    # fix ccode
    commerce['ccode'] = commerce.apply(fix_ccode, axis=1)   
    
    # write to new csv file
    # index = False ensures that there is no Unnamed Column for the index!
    commerce.to_csv(output_file, sep=';', index=False)
    
    print("Written to file {}".format(output_file))

In [97]:
# test run
preprocess(export_file='exp_groups.csv', import_file='imp_groups.csv', output_file='test2.csv')

Written to file test2.csv


In [99]:
test_df = pd.read_csv('test2.csv', sep=';', dtype={'IDNSST': object})

In [100]:
test_df.head()

,year,country,category,IDNSST,ccode,import_value,export_value,balance,exchange
0,2004,Andorra,"Fish,fresh(live or dead),chilled or frozen",034,AND,27.0,0.0,-27.0,27.0
1,2004,Andorra,Other plastics in primary forms,575,AND,0.1,0.0,-0.1,0.1
2,2004,Andorra,Articles of rubber n.e.s.,629,AND,0.1,0.0,-0.1,0.1
3,2004,Andorra,"Knitted or crocheted fabrics,n.e.s.",655,AND,2.9,0.0,-2.9,2.9
4,2004,Andorra,"Made-up articles,wholly or chiefly of textile ...",658,AND,1.3,0.0,-1.3,1.3


In [ ]:
test_df2[(test_df2['year']==2017)&(test_df2['country']=='China')]['export_value'].sum()

## Small refactoring

I do not like the fact that the big **preprocess** function leaves the intermediate files _fixed_export_file.csv_ and _fixed_import_file.csv_ around, but this is a one-off operation and the data isn't going to change more often then once a year, so I guess I could leave it at that.

There should be included some small tests/sanity checks - hardcode some trade values and then make a small test-suite while maintaining approximate equality, bearing in mind the inherent mutability of the trade data.

Anyway, we could also download the original csv files (OG) with requests, but the files are big and heavy and, as I mentioned, they do not change very often, so I would advise against it. 